In [1]:
import torchtext.data as data # 使用torchtext加载文本数据

In [ ]:
# data.Field(sequential=True, tokenize=tokenize, include_lengths=True) / lower=True
	# tokenize = lambda x: x.split()或者def word_cut(text):text = regex.sub(' ', text) return [word for word in jieba.cut(text) if word.strip()], 其中regex = re.compile(r'[^\u4e00-\u9fa5aA-Za-z0-9]')
		# text_field.tokenize = word_cut
		# tokenize: 分词函数, 默认为str.split
		# nltk.word_tokenize
		# data.Field(tokenize='spacy', tokenizer_language='en_core_web_md')
	# include_lengths设为True是为了方便之后使用torch的pack_padded_sequence
	# lower: 是否把数据转化为小写 默认值: False. 
	# sequential: 是否把数据表示成序列，如果是False, 不能使用分词 默认值: True.
	# batch_first: batch作为第一个维度
	# use_vocab: 是否使用词典, 默认为True; 如果为False，那么输入的数据类型必须是数值类型(即使用vocab转换后的)
	# init_token: 文本的初始字符, 默认为None
	# eos_token: 文本的结束字符, 默认为None
	# pad_token: 用于补全的字符, 默认为<pad>
	# unk_token: 替换袋外词的字符，默认为<unk>
	# tensor_type: 把数据转换成的tensor类型 默认值为torch.LongTensor
	# fix_length: 所有样本的长度, 不够则使用pad_token补全, 默认为None, 表示灵活长度
# 标签也可用data.Field定义, 即data.Field(sequential=False, use_vocab=False)
# data.LabelField(sequential=False, use_vocab=False, dtype=torch.long)

In [ ]:
# train, dev, test = data.TabularDataset.splits(
#         path='data', format='tsv', skip_header=True,
#         train='train.tsv', validation='dev.tsv', test='',
#         fields=[
#             ('index', None),
#             ('label', label_field),
#             ('text', text_field)
#         ]
#     )

# 可选format='csv'
# skip_header: 是否跳过文件的第一行
# fields列表存储的Field的顺序必须和csv/tsv文件中每一列的顺序对应, 不需要可添加None
# 如果只想加载训练集, 不给validation和test参数赋值
	# data.TabularDataset.splits返回的是一个元组

In [ ]:
# 除了用表格数据加载, 还可以使用Example, 即from torchtext.data import Example
# Example.fromlist([text,label],fields=fields)  
	# 其中fields同上为列表, text对应一个字符串文本, label为对应的标签(int)
	# 上述为一个example, 最后得到一个examples列表
	# 每个example可以访问它的fields, 如example.comment(为分词后列表)

In [12]:
from torchtext.data import Field,LabelField,Example,Dataset
import torch

corpus = ["D'aww! He matches this background colour. really good, pleace tell him the good news",
         "Yo bitch Ja Rule is more succesful then",
         "If you have a look back at the source",
		 "hello, mary",
		 "hi, nice to meet you"]
labels = [2,3,1,2,0]

TEXT = Field(sequential=True, lower=True, fix_length=10, tokenize=str.split,batch_first=True)
# LABEL = Field(sequential=False, use_vocab=False)
LABEL = LabelField(sequential=False, use_vocab=False, dtype=torch.long)
fields = [("comment", TEXT),("label",LABEL)]

examples = []
for text,label in zip(corpus,labels):
    example = Example.fromlist([text,label],fields=fields)
    examples.append(example)
print(type(examples[0]))
print(examples[0].comment)
print(examples[0].label)

<class 'torchtext.data.example.Example'>
["d'aww!", 'he', 'matches', 'this', 'background', 'colour.', 'really', 'good,', 'pleace', 'tell', 'him', 'the', 'good', 'news']
2


In [ ]:
# 加载完数据可以开始建词表
from torchtext.vocab import Vectors
def load_word_vectors(model_name, model_path):
    vectors = Vectors(name=model_name, cache=model_path) # 如vectors = Vectors(name='glove.6B.300d.txt/glove.840B.300d', cache='.vector_cache')
    return vectors
# vectors = load_word_vectors(pretrained_name, pretrained_path)
# text_field.build_vocab(train_dataset, dev_dataset, vectors=vectors)
    # 或text_field.build_vocab(train_dataset, dev_dataset)
    # 建词表一般是用训练集建，不要用验证集和测试集
    # max_size: 单词表容量
    # unk_init=torch.Tensor.normal_ / 初始化train_data中不存在预训练词向量词表中的单词
# label_field.build_vocab(train_dataset, dev_dataset)

# vocabulary_size = len(text_field.vocab) / class_num = len(label_field.vocab)
    # .vocab.freqs.most_common(20) 数据集里最常出现的20个单词
    # .vocab.itos 列表 index to word
    # .vocab.stoi 字典 word to index
    # pretrained_embeddings = .vocab.vectors
        # model.embedding.weight.data.copy_(pretrained_embeddings)
        # UNK_IDX = .vocab.stoi[field.unk_token]
        # PAD_IDX = .vocab.stoi[field.pad_token]
        # 因为预训练的权重的unk和pad的词向量不是在我们的数据集语料上训练得到的，所以最好置零
        # model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
        # model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
    # embedding_dim = text_field.vocab.vectors.size()[-1]

import torchtext
torchtext.vocab.GloVe(name='840B', dim=300)
torchtext.vocab.FastText(language='en')

In [15]:
new_corpus = [example.comment for example in examples]
new_labels = [example.label for example in examples]
TEXT.build_vocab(new_corpus) # 或者new_corpus = Dataset(examples, fields)
LABEL.build_vocab(new_labels)
print(TEXT.process(new_corpus))
print(TEXT.vocab.stoi)
print(LABEL.vocab.stoi) # {'<unk>': 0, 2: 1, 0: 2, 1: 3, 3: 4} / {2: 0, 0: 1, 1: 2, 3: 3}

print(type(TEXT.vocab.freqs)) # freqs是一个Counter对象, 包含了词表中单词的计数信息
print(TEXT.vocab.freqs['at'])
print(TEXT.vocab.itos[1]) # itos表示index to str
print(TEXT.vocab.stoi['<unk>']) # stoi表示str to index
print(TEXT.vocab.vectors) # 词向量
# TEXT.vocab.load_vectors('fasttext.en.300d') # 自动下载并加载词向量
# print(TEXT.vocab.vectors.shape) torch.Size([25, 300])

# p = os.path.expanduser("vector_cache/sgns.wiki.bigram-char") # 加载本地词向量
# TEXT.vocab.load_vectors(vocab.Vectors(p))

tensor([[10, 14, 23, 35,  7,  9, 29, 12, 28, 33],
        [37,  8, 20, 30, 19, 25, 32, 34,  1,  1],
        [18,  3, 13,  4, 21,  6,  5,  2, 31,  1],
        [15, 22,  1,  1,  1,  1,  1,  1,  1,  1],
        [16, 27, 36, 24,  3,  1,  1,  1,  1,  1]])
defaultdict(<function _default_unk_index at 0x7f642d82b3b0>, {'<unk>': 0, '<pad>': 1, 'the': 2, 'you': 3, 'a': 4, 'at': 5, 'back': 6, 'background': 7, 'bitch': 8, 'colour.': 9, "d'aww!": 10, 'good': 11, 'good,': 12, 'have': 13, 'he': 14, 'hello,': 15, 'hi,': 16, 'him': 17, 'if': 18, 'is': 19, 'ja': 20, 'look': 21, 'mary': 22, 'matches': 23, 'meet': 24, 'more': 25, 'news': 26, 'nice': 27, 'pleace': 28, 'really': 29, 'rule': 30, 'source': 31, 'succesful': 32, 'tell': 33, 'then': 34, 'this': 35, 'to': 36, 'yo': 37})
defaultdict(<function _default_unk_index at 0x7f642d82b3b0>, {2: 0, 0: 1, 1: 2, 3: 3})
<class 'collections.Counter'>
1
<pad>
0
None


In [ ]:
# 用torchtext的迭代器来批量加载数据，torchtext.data里的BucketIterator非常好用
	# data.BucketIterator.splits不但返回的是元组，它的参数datasets要求也是以元组形式
# 它可以把长度相近的文本数据尽量都放到一个batch里，这样最大程度地减少padding
# 数据就少了很多无意义的0，也减少了矩阵计算量
# train_iter, dev_iter = data.Iterator.splits(
#         (train_dataset, dev_dataset),
#         batch_sizes=(args.batch_size, len(dev_dataset)),
#         sort_key=lambda x: len(x.text),
#         device=args.device, repeat=False, shuffle=True)
# 参数
# sort_within_batch设为True的话, 一个batch内的数据就会按sort_key的排列规则降序排列
# sort_key是排列的规则
# batch_size

# for batch in train_iter: batch.text, batch.label, batch.batch_size
# len(data_iter.dataset)
# 若inclue_lengths为True, 则还会包含一个句子长度的Tensor, 即返回(seq_length, batch_size)和(batch_size, )

In [ ]:
from torchtext.datasets import WikiText2, WikiText103
# WikiText103.splits(root=root, text_field=TEXT) # TEXT/data.Field

from torchtext.datasets import IMDB

train_data, test_data = IMDB.splits(TEXT, LABEL)

print('len of train data:', len(train_data))        # 25000
print('len of test data:', len(test_data))          # 25000

# torchtext.data.Example : 用来表示一个样本，数据+标签
print(train_data.examples[15].text)                 # 文本：句子的单词列表
print(train_data.examples[15].label)                # 标签: 积极

import random
SEED = 1234
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

TEXT.build_vocab(train_data, max_size=10000, vectors='glove.6B.100d') # unk_init=torch.Tensor.normal_
LABEL.build_vocab(train_data)

print(len(TEXT.vocab))             # 10002
print(TEXT.vocab.itos[:12])        # ['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is', 'in', 'I']
print(TEXT.vocab.stoi['and'])      # 5
print(LABEL.vocab.stoi)            # defaultdict(None, {'neg': 0, 'pos': 1})
print(TEXT.vocab.freqs.most_common(20))
